# Day 10 - Part 1

In [144]:
def display(data):
    for l in data:
        for k, v in l.items():
            print(k.ljust(7), v)
        print()

def get_table(button, l):
    return tuple([1 if i in button else 0 for i in range(l)])

def xor_t(t1: tuple, t2: tuple):
    return tuple(a ^ b for a, b in zip(t1, t2))

def convert_b2(i, n):
    return tuple([int(d) for d in f"{i:0{n}b}"])

def count_up(n):
    return [convert_b2(i, n) for i in range(2 ** n)]

def toggle(pattern, buttons, t_pattern):
    to_toggle = [b[0] for b in zip(buttons, t_pattern) if b[1]]
    for b in to_toggle:
        pattern = xor_t(pattern, b)

    return pattern

with open("manual_t.txt", 'r',  encoding='utf-8') as f:
    data = [
        {
            'pattern': tuple(map(lambda x: 1 if x == '#' else 0, p[0].strip('[]'))), 
            'buttons': [list(map(int, x.strip('()').split(','))) for x in p[1:-1]], 
            'jolts': list(map(int, p[-1].strip('{}').split(',')))
        }
        for line in f 
        for p in [line.strip().split()]
    ]

for d in data:
    for i, b in enumerate(d["buttons"]):
        d['buttons'][i] = get_table(b, len(d["pattern"]))

total = 0
for d in data:
    depth = float('inf')
    for toggle_pattern in count_up(len(d['buttons'])):
        flipped_pattern = toggle(d['pattern'], d['buttons'], toggle_pattern)
        if not any(flipped_pattern):
            depth = min(depth, len([x for x in toggle_pattern if x]))
            if depth == 1:
                break

    total += depth

display(data)

total


pattern (0, 1, 1, 0)
buttons [(0, 0, 0, 1), (0, 1, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (1, 0, 1, 0), (1, 1, 0, 0)]
jolts   [3, 5, 4, 7]

pattern (0, 0, 0, 1, 0)
buttons [(1, 0, 1, 1, 1), (0, 0, 1, 1, 0), (1, 0, 0, 0, 1), (1, 1, 1, 0, 0), (0, 1, 1, 1, 1)]
jolts   [7, 5, 12, 7, 2]

pattern (0, 1, 1, 1, 0, 1)
buttons [(1, 1, 1, 1, 1, 0), (1, 0, 0, 1, 1, 0), (1, 1, 1, 0, 1, 1), (0, 1, 1, 0, 0, 0)]
jolts   [10, 11, 11, 5, 10, 5]



7

# Better Part 1 - sub 100ms

In [ ]:
import time
from itertools import combinations

class DoNum:
    def __init__(self, entry: int | tuple | list, width: int = None):
        if isinstance(entry, tuple | list):
            self.width = len(entry)
            self.bits = entry
            self.numb = self.convert_bits(entry)
        elif isinstance(entry, int):
            self.width = width if width else entry.bit_length()
            self.numb = entry
            self.bits = None
        else:
            raise "something went terribly wrong"
        
    @property
    def bits_tuple(self):
        if self.bits is None:
            self.bits = self.convert_num(self.numb)
        return self.bits

    def convert_num(self, num):
        return tuple(map(int, f'{num:0{self.width}b}'))

    def convert_bits(self, bits):
        p_int = 0
        for b in bits:
            p_int = (p_int << 1) | b
        return p_int

    def __iter__(self):
        return iter(self.bits)

    def __len__(self):
        return self.width

    def __xor__(self, other):
        val = NotImplemented

        if isinstance(other, DoNum):
            val = self.numb ^ other.numb
        if isinstance(other, int):
            val = self.numb ^ other
        if isinstance(other, tuple | list):
            val = self.numb ^ self.convert_bits(other)

        if val is NotImplemented:
            return NotImplemented
        
        return DoNum(val, width=self.width)

    def __rxor__(self, other):
        return self.__xor__(other)

    def __repr__(self):
        return f"DoNum({self.numb}|{self.numb:0{self.width}b})"


def get_table(button, l):
    return tuple([1 if i in button else 0 for i in range(l)])

def toggle_sorted(n):
    for r in range(n + 1):
        for indices in combinations(range(n), r):
            yield indices

def read_process():
    start = time.perf_counter()
    with open("manual.txt", 'r',  encoding='utf-8') as f:
        data = [
            {
                'pattern': tuple(map(lambda x: 1 if x == '#' else 0, p[0].strip('[]'))), 
                'buttons': [list(map(int, x.strip('()').split(','))) for x in p[1:-1]], 
                'jolts': list(map(int, p[-1].strip('{}').split(',')))
            }
            for line in f 
            for p in [line.strip().split()]
        ]

    for d in data:
        d['pattern'] = DoNum(d['pattern'])
        for i, b in enumerate(d["buttons"]):
            d['buttons'][i] = DoNum(get_table(b, len(d["pattern"])))

    total = 0
    for d in data:
        n_buttons = len(d['buttons'])

        for indices in toggle_sorted(n_buttons):
            current = d['pattern']

            for idx in indices:
                current = current ^ d['buttons'][idx]

            if current.numb == 0:
                total += len(indices)
                break

    time_taken = time.perf_counter() - start
    return total, time_taken

total_time = 0
N_TIME = 100
for i in range(N_TIME):
    total, tt = read_process()
    total_time += tt

print("total:", total, "time:", f"{round(total_time / N_TIME * 1000, 1)}ms")

total: 481 time: 73.6ms


# Part 2